# DRIADA INTENSE: Quick Start Tutorial (5 minutes)

Welcome to DRIADA's INTENSE module! This notebook will get you analyzing neuronal selectivity in just 5 minutes.

## What is INTENSE?

INTENSE (Information-Theoretic Evaluation of Neuronal Selectivity) helps you discover which neurons encode specific behavioral or environmental variables using mutual information analysis.

Key advantages:
- Detects both linear AND nonlinear relationships
- Rigorous statistical testing with multiple comparison correction
- Handles temporal delays between neural activity and behavior
- Works with both discrete (e.g., trial type) and continuous (e.g., position) variables

## Setup

First, let's import DRIADA and set up our environment:

In [ ]:
# Add DRIADA to Python path
import sys
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import required libraries
import driada
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

print(f"DRIADA version: {driada.__version__}")
print("Setup complete! ✓")

## Step 1: Generate Synthetic Neural Data

For this demo, we'll use DRIADA's synthetic data generator to create an experiment with:
- 20 neurons (some will be selective to specific features)
- 2 discrete features (like trial type: left/right)
- 2 continuous features (like x,y position)
- 5 minutes of recording

In [ ]:
# Generate synthetic experiment
exp = driada.generate_synthetic_exp(
    n_dfeats=2,      # discrete features (e.g., trial type)
    n_cfeats=2,      # continuous features (e.g., x, y position)  
    nneurons=20,     # number of neurons
    duration=300,    # 5 minutes recording
    seed=42          # reproducible results
)

print("Created experiment with:")
print(f"  • {exp.n_cells} neurons")
print(f"  • {exp.n_frames} timepoints")
print(f"  • Features: {list(exp.dynamic_features.keys())}")

## Step 2: Run INTENSE Analysis

Now let's discover which neurons are selective to which features using mutual information:

In [ ]:
# Analyze neuronal selectivity
stats, significance, info, results = driada.compute_cell_feat_significance(
    exp,
    mode='two_stage',         # efficient two-stage testing
    n_shuffles_stage1=50,     # preliminary screening
    n_shuffles_stage2=1000,   # rigorous validation
    verbose=True              # show progress
)

print("\nAnalysis complete! ✓")

## Step 3: Extract and Visualize Results

Let's see what INTENSE discovered:

In [ ]:
# Get significant neuron-feature pairs
significant_neurons = exp.get_significant_neurons()
total_pairs = sum(len(features) for features in significant_neurons.values())

print(f"Found {len(significant_neurons)} neurons with significant selectivity")
print(f"Total significant neuron-feature pairs: {total_pairs}\n")

# Show detailed results for first 3 neurons
if significant_neurons:
    print("Significant relationships:")
    for i, (cell_id, features) in enumerate(significant_neurons.items()):
        if i >= 3:  # Show first 3
            break
        for feat_name in features:
            pair_stats = exp.get_neuron_feature_pair_stats(cell_id, feat_name)
            
            print(f"\nNeuron {cell_id} ↔ Feature '{feat_name}':")
            print(f"  • Mutual Information: {pair_stats['pre_rval']:.4f}")
            if 'pval' in pair_stats and pair_stats['pval'] is not None:
                print(f"  • P-value: {pair_stats['pval']:.2e}")
            print(f"  • Optimal delay: {pair_stats.get('shift_used', 0):.2f}s")
else:
    print("No significant relationships found in this synthetic data.")

## Step 4: Visualize a Neuron-Feature Relationship

Let's visualize how a selective neuron responds to its preferred feature:

In [ ]:
# Visualize the first significant relationship
if significant_neurons:
    # Get first significant pair
    cell_id = list(significant_neurons.keys())[0]
    feat_name = significant_neurons[cell_id][0]
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    driada.intense.plot_neuron_feature_pair(exp, cell_id, feat_name, ax=ax)
    plt.title(f"Neuron {cell_id} selectivity to {feat_name}")
    plt.tight_layout()
    plt.show()
else:
    print("No significant relationships to visualize.")

## Understanding the Results

### What do these values mean?

- **Mutual Information (MI)**: Measures how much information the neural activity provides about the behavioral variable
  - 0 = no relationship
  - Higher values = stronger relationship
  - Captures both linear and nonlinear dependencies

- **P-value**: Statistical significance after correcting for multiple comparisons
  - < 0.01 = highly significant
  - Uses rigorous shuffling-based null distribution

- **Optimal delay**: Time shift that maximizes the relationship
  - Positive = neural activity follows behavior
  - Negative = neural activity precedes behavior
  - Accounts for calcium indicator dynamics (~1-2s delay)

### The Two-Stage Testing Approach

INTENSE uses an efficient two-stage procedure:
1. **Stage 1**: Quick screening with 50-100 shuffles to reject obviously non-significant pairs
2. **Stage 2**: Rigorous testing with 1000+ shuffles for candidates that pass Stage 1

This saves ~100x computation time while maintaining statistical rigor!

## Next Steps

Congratulations! You've just completed your first INTENSE analysis. 🎉

To learn more:
1. **Notebook 02**: Understanding INTENSE results in depth
2. **Notebook 03**: Working with your own neural data
3. **Examples folder**: See advanced features like mixed selectivity analysis

### Using Your Own Data

Here's a template for analyzing your data:

```python
# Your neural recordings
calcium_traces = np.array(...)  # shape: (n_neurons, n_timepoints)

# Your behavioral variables
dynamic_features = {
    'position_x': np.array(...),      # continuous
    'position_y': np.array(...),      # continuous
    'trial_type': np.array(...),      # discrete
    'reward': np.array(...)           # discrete
}

# Create experiment
exp = driada.Experiment(
    signature='MyExperiment',
    calcium=calcium_traces,
    dynamic_features=dynamic_features,
    static_features={'fps': 20.0}
)

# Run INTENSE
results = driada.compute_cell_feat_significance(exp)
```

Happy analyzing! 🧠✨